In [3]:
import pandas as pd

df = pd.read_csv('http://www.ppgia.pucpr.br/~jean.barddal/datascience/enron.csv')



In [4]:
df.head(5)

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,name
0,4175000.0,2869717.0,-3081055.0,NaN,phillip.allen@enron.com,1729541.0,13868.0,2195.0,47.0,65.0,NaN,304805.0,152.0,False,126027.0,-126027.0,201955.0,1407.0,2902.0,4484442.0,1729541.0,ALLEN PHILLIP K
1,NaN,178980.0,NaN,NaN,NaN,257817.0,3486.0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,182466.0,257817.0,BADUM JAMES P
2,NaN,NaN,-5104.0,NaN,james.bannantine@enron.com,4046157.0,56301.0,29.0,39.0,0.0,NaN,NaN,864523.0,False,1757552.0,-560222.0,477.0,465.0,566.0,916197.0,5243487.0,BANNANTINE JAMES M
3,1200000.0,1295738.0,-1386055.0,NaN,NaN,6680544.0,11200.0,NaN,NaN,NaN,NaN,1586055.0,2660303.0,False,3942714.0,NaN,267102.0,NaN,NaN,5634343.0,10623258.0,BAXTER JOHN C
4,400000.0,260455.0,-201641.0,NaN,frank.bay@enron.com,NaN,129142.0,NaN,NaN,NaN,NaN,NaN,69.0,False,145796.0,-82782.0,239671.0,NaN,NaN,827696.0,63014.0,BAY FRANKLIN R


In [14]:
# Método de Tukey

# 1) passo: obter colunas númericas
# 2) passo: Criar uma lista para os outlier identificados durante o processo
# 3) passo: um laço de repeticao, incrementando a lista acima de outliers a cada variavel
# 4) passo: Analisar os outliers mais frequentes



# todos os indices marcados como outliers
outliers_global = []

# laco de repitacao por coluna
for col in df.select_dtypes(include="number").columns.values:


  #calculando o método de tukey para a coluna col


  #calculando os quartis (Q3 e Q1)
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)

  # Calculando o intervalo inter-quartis
  iqr = q3 - q1

  #Calculando limites superiores e inferiores
  sup = q3 + 1.5 * iqr
  inf = q1 - 1.5 * iqr

  #Selecionando valores externos ao intervalo (i.e.,>sup ou < inf)
  ix_outliers_col = list(df[(df[col] > sup) | (df[col] < inf)].index)

  #adicionando os indices desta coluna a lista global
  outliers_global.extend(ix_outliers_col)
  #for ix in ix_outliers_col:
    #outliers_global.append(ix)

#  print(outliers_global)
  from collections import Counter   
  for ix, contagem in Counter(outliers_global).most_common(10):     
    print(f"{ix} \t {contagem} \t {df.loc[ix]['name']} ")


0 	 1 	 ALLEN PHILLIP K 
7 	 1 	 BELDEN TIMOTHY N 
31 	 1 	 DELAINEY DAVID W 
42 	 1 	 FALLON JAMES B 
75 	 1 	 KITCHEN LOUISE 
78 	 1 	 LAVORATO JOHN J 
79 	 1 	 LAY KENNETH L 
91 	 1 	 MCMAHON JEFFREY 
122 	 1 	 SKILLING JEFFREY K 
130 	 1 	 TOTAL 
0 	 2 	 ALLEN PHILLIP K 
130 	 2 	 TOTAL 
7 	 1 	 BELDEN TIMOTHY N 
31 	 1 	 DELAINEY DAVID W 
42 	 1 	 FALLON JAMES B 
75 	 1 	 KITCHEN LOUISE 
78 	 1 	 LAVORATO JOHN J 
79 	 1 	 LAY KENNETH L 
91 	 1 	 MCMAHON JEFFREY 
122 	 1 	 SKILLING JEFFREY K 
0 	 3 	 ALLEN PHILLIP K 
130 	 3 	 TOTAL 
7 	 2 	 BELDEN TIMOTHY N 
47 	 2 	 FREVERT MARK A 
31 	 1 	 DELAINEY DAVID W 
42 	 1 	 FALLON JAMES B 
75 	 1 	 KITCHEN LOUISE 
78 	 1 	 LAVORATO JOHN J 
79 	 1 	 LAY KENNETH L 
91 	 1 	 MCMAHON JEFFREY 
130 	 4 	 TOTAL 
0 	 3 	 ALLEN PHILLIP K 
7 	 2 	 BELDEN TIMOTHY N 
47 	 2 	 FREVERT MARK A 
31 	 1 	 DELAINEY DAVID W 
42 	 1 	 FALLON JAMES B 
75 	 1 	 KITCHEN LOUISE 
78 	 1 	 LAVORATO JOHN J 
79 	 1 	 LAY KENNETH L 
91 	 1 	 MCMAHON JEFFREY 
130 	 

In [19]:
from sklearn.ensemble import IsolationForest

# selecionando apenas colunas numericas
df_num = df.select_dtypes(include='number')

# imputando base de dados com a media
df_num.fillna(df_num.mean(),inplace=True)

# criar o modelo de isolation Forest
isof = IsolationForest(random_state=101)
isof.fit(df_num)


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


IsolationForest(behaviour='deprecated', bootstrap=False, contamination='auto',
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=101, verbose=0, warm_start=False)

In [24]:
# Predicao (inlier ou outlier)
df['inlier'] = isof.predict(df_num)


In [23]:
#mostrando os outliers
df[df['inlier'] == -1]

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,name,inlier
47,2000000.0,6426990.0,-3367011.0,NaN,mark.frevert@enron.com,10433518.0,86987.0,21.0,242.0,6.0,2000000.0,1617011.0,7427621.0,False,4188667.0,NaN,1060932.0,2979.0,3275.0,17252530.0,14622185.0,FREVERT MARK A,-1
73,1000000.0,NaN,NaN,NaN,steven.kean@enron.com,2022048.0,41953.0,6759.0,140.0,387.0,NaN,300000.0,1231.0,False,4131594.0,NaN,404338.0,3639.0,12754.0,1747522.0,6153642.0,KEAN STEVEN J,-1
78,8000000.0,NaN,NaN,NaN,john.lavorato@enron.com,4158995.0,49537.0,2585.0,528.0,411.0,NaN,2035380.0,1552.0,False,1008149.0,NaN,339288.0,3962.0,7259.0,10425757.0,5167144.0,LAVORATO JOHN J,-1
79,7000000.0,202911.0,-300000.0,NaN,kenneth.lay@enron.com,34348384.0,99832.0,36.0,123.0,16.0,81525000.0,3600000.0,10359729.0,True,14761694.0,NaN,1072321.0,2411.0,4273.0,103559793.0,49110078.0,LAY KENNETH L,-1
130,97343619.0,32083396.0,-27992891.0,1398517.0,NaN,311764000.0,5235198.0,NaN,NaN,NaN,83925000.0,48521928.0,42667589.0,False,130322299.0,-7576788.0,26704229.0,NaN,NaN,309886585.0,434509511.0,TOTAL,-1


In [ ]:
isof.score_samples(df_num)